# Aviation Accident Analysis

## Overview
This project analyzes aviation accident data to identify risk factors affecting severity, aircraft damage, and injuries. This insights will help aviation authorities and manufacturers improve safety protocols, training, and maintenance practices.

## Problem Statement
Aviation accidents pose significant risks to passengers, crew, and aircraft manufacturers. By analyzing historical accident data, I aim to:
1. Determine how weather conditions influence the severity of injuries.
2. Identify aircraft manufacturers with a high proportion of serious but non-fatal accidents.
3. Assess the impact of different flight phases on total aircraft damage.

## Data Understanding
The dataset contains aviation accident records, including information on aircraft type, manufacturer, flight phase, weather conditions, and accident outcomes. Key variables include:
- `Event Date`: Date of the accident.
- `Aircraft Manufacturer`: Company that produced the aircraft.
- `Injury Severity`: Classification of injuries (None, Minor, Serious, Fatal).
- `Broad Phase of Flight`: Phase of flight at the time of the accident.
- `Weather Condition`: Weather conditions (VMC - Visual Meteorological Conditions, IMC - Instrument Meteorological Conditions).

## Data Preparation

In [199]:
# Importing the necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.set_style("whitegrid")
sns.set_context('talk')

In [200]:
# Loading the dataset
df = pd.read_csv("AviationData.csv", encoding="latin1", low_memory=False)
df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


### Understanding the dataset

In [201]:
# Inspecting the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52790 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87572 non-null  object 
 14  Make                    88826 non-null

- The dataset has 88,889 rows and 31 columns
- Several columns have missing values (eg. `` Aircraft.damage ``, `` Aircraft.Category `` etc)

In [202]:
# show the total number of missing values in each column
df.isna().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     52
Country                     226
Latitude                  54507
Longitude                 54516
Airport.Code              38640
Airport.Name              36099
Injury.Severity            1000
Aircraft.damage            3194
Aircraft.Category         56602
Registration.Number        1317
Make                         63
Model                        92
Amateur.Built               102
Number.of.Engines          6084
Engine.Type                7077
FAR.Description           56866
Schedule                  76307
Purpose.of.flight          6192
Air.carrier               72241
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Weather.Condition          4492
Broad.phase.of.flight     27165
Report.Status              6381
Publication.Date          13771
dtype: i

## Data Cleaning and Normalization
Unnecessary columns will be removed, and missing or inconsistent values will be handled to ensure data quality.

In [203]:
# gives the columns names in the dataset
df.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status',
       'Publication.Date'],
      dtype='object')

In [204]:
# Keep only relevant columns
columns_to_keep = ['Aircraft.damage', 'Make', 'Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Weather.Condition', 'Broad.phase.of.flight']
df = df[columns_to_keep]
df.head()

,Aircraft.damage,Make,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Weather.Condition,Broad.phase.of.flight
0,Destroyed,Stinson,2.0,0.0,0.0,UNK,Cruise
1,Destroyed,Piper,4.0,0.0,0.0,UNK,Unknown
2,Destroyed,Cessna,3.0,NaN,NaN,IMC,Cruise
3,Destroyed,Rockwell,2.0,0.0,0.0,IMC,Cruise
4,Destroyed,Cessna,1.0,2.0,NaN,VMC,Approach


In [205]:
# several columns have missing values 
# total number of missing values in each column
df.isna().sum()

Aircraft.damage            3194
Make                         63
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Weather.Condition          4492
Broad.phase.of.flight     27165
dtype: int64

- From the missing values in the columns above, I will remove the missing values in the columns ``Aircraft.damage`` , ``Make`` , and `` Weather.Condition``
- I will need to replace the columns ``Total.Fatal.Injuries``, ``Total.Serious.Injuries`` , ``Total.Minor.Injuries `` and ``Broad.phase.of.flight``

### Dealing with missing values

In [206]:
# remove the missing values in the 3 columns
df.dropna(subset=["Aircraft.damage", "Make", "Weather.Condition"],inplace=True)
df.head()

,Aircraft.damage,Make,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Weather.Condition,Broad.phase.of.flight
0,Destroyed,Stinson,2.0,0.0,0.0,UNK,Cruise
1,Destroyed,Piper,4.0,0.0,0.0,UNK,Unknown
2,Destroyed,Cessna,3.0,NaN,NaN,IMC,Cruise
3,Destroyed,Rockwell,2.0,0.0,0.0,IMC,Cruise
4,Destroyed,Cessna,1.0,2.0,NaN,VMC,Approach


In [207]:
# rechecking whether they have been removed
df.isna().sum()

Aircraft.damage               0
Make                          0
Total.Fatal.Injuries      10703
Total.Serious.Injuries    11845
Total.Minor.Injuries      11114
Weather.Condition             0
Broad.phase.of.flight     22124
dtype: int64

### Replacing the remaining columns 

In [208]:
# Replace the Broad.phase.of.flight column with Unknown 
# I want to assume for the missing values, the phase of flight was unknown 
df["Broad.phase.of.flight"].fillna("Unknown", inplace=True)
df.head()

,Aircraft.damage,Make,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Weather.Condition,Broad.phase.of.flight
0,Destroyed,Stinson,2.0,0.0,0.0,UNK,Cruise
1,Destroyed,Piper,4.0,0.0,0.0,UNK,Unknown
2,Destroyed,Cessna,3.0,NaN,NaN,IMC,Cruise
3,Destroyed,Rockwell,2.0,0.0,0.0,IMC,Cruise
4,Destroyed,Cessna,1.0,2.0,NaN,VMC,Approach


### Standardizing the ``Weather.Condition`` and ``Make`` column

In [209]:
# check the unique weather conditions
df['Weather.Condition'].unique()

array(['UNK', 'IMC', 'VMC', 'Unk'], dtype=object)

In [210]:
# Standardize weather condition labels
df['Weather.Condition'] = df['Weather.Condition'].replace({'Unk': 'UNK'})

In [211]:
df['Weather.Condition'].unique()

array(['UNK', 'IMC', 'VMC'], dtype=object)

- The `Weather.Condition` column names have been standardized

In [212]:
# View unique values
df['Make'].value_counts() 

Cessna                   21949
Piper                    11874
CESSNA                    4283
Beech                     4245
PIPER                     2515
                         ...  
Marshall                     1
HELICYCLE/MORRISEY RR        1
Mullinax                     1
GRUMMAN SCHWEIZER            1
Flying K Enterprises         1
Name: Make, Length: 7965, dtype: int64

- The `Make` column has values such as Cessna and CESSNA that mean the same

In [213]:
# fix inconsistencies
df['Make'] = df['Make'].str.capitalize() 

# recheck
df['Make'].value_counts()

Cessna               26232
Piper                14389
Beech                 5142
Bell                  2482
Mooney                1290
                     ...  
Mchugh tony              1
Greene r/greene s        1
Mk ii                    1
Gillespie/pitts          1
Hanson lonn              1
Name: Make, Length: 7364, dtype: int64

### Dealing with the Injuries columns

In [214]:
# Convert the columns to numeric
df['Total.Fatal.Injuries'] = pd.to_numeric(df['Total.Fatal.Injuries'], errors='coerce').astype('Int64')
df['Total.Serious.Injuries'] = pd.to_numeric(df['Total.Serious.Injuries'], errors='coerce').astype('Int64')
df['Total.Minor.Injuries'] = pd.to_numeric(df['Total.Minor.Injuries'], errors='coerce').astype('Int64')

# Replace the missing values in the Injuries columns with 0 (to mean maybe there were no injuries)
df[['Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries']] = df[['Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries']].fillna(0)
df.head()

,Aircraft.damage,Make,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Weather.Condition,Broad.phase.of.flight
0,Destroyed,Stinson,2,0,0,UNK,Cruise
1,Destroyed,Piper,4,0,0,UNK,Unknown
2,Destroyed,Cessna,3,0,0,IMC,Cruise
3,Destroyed,Rockwell,2,0,0,IMC,Cruise
4,Destroyed,Cessna,1,2,0,VMC,Approach


In [215]:
# Recheck for any missing values
df.isna().sum()

Aircraft.damage           0
Make                      0
Total.Fatal.Injuries      0
Total.Serious.Injuries    0
Total.Minor.Injuries      0
Weather.Condition         0
Broad.phase.of.flight     0
dtype: int64

In [216]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82336 entries, 0 to 88886
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Aircraft.damage         82336 non-null  object
 1   Make                    82336 non-null  object
 2   Total.Fatal.Injuries    82336 non-null  Int64 
 3   Total.Serious.Injuries  82336 non-null  Int64 
 4   Total.Minor.Injuries    82336 non-null  Int64 
 5   Weather.Condition       82336 non-null  object
 6   Broad.phase.of.flight   82336 non-null  object
dtypes: Int64(3), object(4)
memory usage: 5.3+ MB


- Now the dataset is clean with no missing values
- The dataset has 82336 rows and 7 columns